In [1]:
import csv
import pandas as pd
import numpy as np
import scipy.linalg as linalg
from hodgerank_tools import *

In [2]:
symptoms = ['A', 'B', 'C','D']
#covid_data = np.array([symptoms, [3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]])
covid_df = pd.DataFrame(np.array([[3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]]), columns=symptoms)

In [3]:
symptoms = ['A', 'B', 'C','D']
my_data = df_to_dict_list(covid_df)
print(my_data)

[{'A': 3.0, 'B': 2.0, 'C': 2.0, 'D': 5.0}, {'A': 7.0, 'B': 8.0, 'C': 9.0}, {'A': 2.0, 'B': 2.0, 'C': 1.0, 'D': 3.0}]


In [4]:
my_nodes = get_nodes(my_data)
print(my_nodes)

['A', 'B', 'C', 'D']


In [5]:
get_f_W(my_data, my_nodes)

(array([0. , 0. , 1.5, 0. , 2. , 2.5]),
 array([[3., 0., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0.],
        [0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 0., 2.]]))

In [6]:
print(rank(my_data))

(  node         r
 0    D  1.500000
 1    A -0.409091
 2    B -0.500000
 3    C -0.590909,
 0.38429752066115663)

In [7]:
print(naive_rank_0(my_data))

{'A': 4.0, 'B': 4.0, 'C': 4.0, 'D': 4.0}
  node    r
0    A  4.0
1    B  4.0
2    C  4.0
3    D  4.0
